In [1]:
from docs_preprocess import DocumentProcessor
from VecStore import VectorStore
from langchain.llms import HuggingFacePipeline
from pathlib import Path


In [2]:
from typing import List, Optional, Any
from langchain_core.language_models.llms import LLM

class DeepSeekLLM(LLM):
    model: Any
    tokenizer: Any
    max_new_tokens: int = 512
    temperature: float = 0.0           # 默认给 RAG 用

    @property
    def _llm_type(self) -> str:
        return "deepseek_hf"

    def _call(self, prompt: str,
              stop: Optional[List[str]] = None,
              **kwargs) -> str:
        # 1. 把普通 prompt 包装进官方 chat_template
        messages = [{"role": "user", "content": prompt}]
        input_ids = self.tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(self.model.device)

        # 2. 生成
        outputs = self.model.generate(
            input_ids,
            max_new_tokens=kwargs.get("max_new_tokens", self.max_new_tokens),
            temperature=kwargs.get("temperature", self.temperature),
        )
        text = self.tokenizer.decode(
            outputs[0][input_ids.shape[1]:],
            skip_special_tokens=True,
        )

        # 3. 手动处理 stop tokens
        if stop:
            for s in stop:
                if s in text:
                    text = text.split(s)[0]

        return text.strip()


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

model_dir = "/home/lyus4/yuheng/All_in_LLM/deepseek-llm-7b-chat"
tok   = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_dir,
                                             trust_remote_code=True,
                                             torch_dtype="auto",
                                             device_map="auto")



/home/lyus4/anaconda3/envs/rag_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-14 10:55:59,163] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/lyus4/anaconda3/envs/rag_env/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/lyus4/anaconda3/envs/rag_env/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.85s/it]
We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.


In [20]:
llm = DeepSeekLLM(model=model, tokenizer=tok, max_new_tokens=512, temperature=0.2)
print(llm("你好，请自我介绍一下。"))

/home/lyus4/anaconda3/envs/rag_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


你好！我是一个人工智能助手，名为DeepSeek Chat。我由中国的DeepSeek团队开发，旨在帮助用户解答问题、提供信息和执行各种任务。我能够处理多种主题，包括但不限于科学、数学、历史、文化、技术等。如果你有任何问题，欢迎随时向我提问。


In [21]:
print(llm("In Wireless communication, what is RB"))

/home/lyus4/anaconda3/envs/rag_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


In wireless communication, RB stands for Resource Block. It is a basic unit of data in the GSM (Global System for Mobile communication) and UMTS (Universal Mobile Telecommunication System) standards. A Resource Block contains information about the data to be transmitted, such as the data itself, error-correction codes, and synchronization information.

Resource Blocks are used to encapsulate and transmit data in the air interface of wireless communication systems. They are also used to allocate resources, such as bandwidth and power, to different users or services.

In GSM, a Resource Block is typically 128 bytes in size, while in UMTS, it can be up to 64 KB. The size of a Resource Block can vary depending on the specific requirements of the communication system and the type of data being transmitted.


IndexError: list assignment index out of range

In [22]:
# 路径配置
MODEL_PATH =  "/home/lyus4/yuheng/All_in_LLM/all-MiniLM-L6-v2"
INDEX_PATH = Path("../vector_store/faiss_ivfflat_100")  # 无需加 .index 后缀
VECTORS_PATH = Path("../index/vectors.npy")
DOC_JSON_PATH = Path("output_chunks.json")

In [23]:
vs = VectorStore(model_path=MODEL_PATH, db_path=INDEX_PATH)
vs.load_documents_and_metadata(json_path=DOC_JSON_PATH)
vs.describe()


[INFO] 初始化 VectorStore -> ../vector_store/faiss_ivfflat_100
[INFO] 索引加载成功: ../vector_store/faiss_ivfflat_100
[INFO] 类型: IndexIVFFlat, 维度: 384, 数量: 18343
[INFO] 加载文档 18343 条

[INFO] VectorStore 状态描述：
- 文档数: 18343
- 向量数: 0
- 索引类型: IndexIVFFlat
- 向量维度: 384
- 向量总数: 18343


In [24]:

results = vs.search("what is OFDMA?", k=5, score_mode="reciprocal")

No sentence-transformers model found with name /home/lyus4/yuheng/All_in_LLM/all-MiniLM-L6-v2. Creating a new one with mean pooling.


[INFO] 嵌入模型已加载


In [25]:
results

[('Orthogonal frequency division multiple access (OFDMA), which is the multi-user\nversion of OFDM that was discussed in Section 9.12. In OFDMA multiple access is\naccomplished through the assignment of subchannels (subcarriers) to individual users.',
  0.6585590018011712,
  {'document': '../documents/Digital-Communication-Systems.pdf',
   'page': 587,
   'chunk_idx': 5}),
 ('Chapter\nSignaling over Fading Channels\nNaturally, OFDMA inherits the distinctive features of OFDM. In particular, OFDMA\nis well suited for high data-rate transmissions over delay-dispersive channels, realized\nby exploiting the principle of divide and conquer. Accordingly, OFDMA is\ncomputationally efficient in using the FFT algorithm. Moreover, OFDMA lends itself\nto the combined use of MIMO, hence the ability to improve spectral efficiency and\ntake advantage of channel flexibility.\nCode-division multiple access (CDMA), which distinguishes itself by exploiting the\nunderlying principle of spread spectrum sig

In [40]:
from langchain_core.prompts import PromptTemplate
from utils import format_context_grouped, expand_acronyms
# 定义 Prompt 模板

template_test = """
<Role>
You are a 5G wireless communication expert.

<Goal>
Answer the question using the information in the context below.
If the context is insufficient, reply exactly: **"I don't know"**.

<Context>
{context}

<Question>
{question}

<Instructions>
1. Explain simply and clearly, as if to a non-expert.  
2. Use the following **answer format strictly**:  
   - Start with a 1-2 sentence summary.  
   - Then, list the explanation in **numbered bullet points** (1., 2., 3., etc.).  
3. Give the reference.
4. Conform to 3GPP protocol.

<Answer>

"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template_test
)

# 执行检索
question = "What is beam management?"
question = expand_acronyms(question)

top_k_results = vs.search(question, k=3, score_mode="reciprocal")
context = format_context_grouped(top_k_results, with_metadata=True, with_score=True)

# 构造 Prompt 输入
prompt_input = prompt.format(context=context, question=question)

# 输出 prompt（可传入 LLM）
print(prompt_input)





<Role>
You are a 5G wireless communication expert.

<Goal>
Answer the question using the information in the context below.
If the context is insufficient, reply exactly: **"I don't know"**.

<Context>
[Document: ../documents/3GPP_38/38214-i60.docx, Page: 1]
(score=0.495) to the UE reported threshold beamSwitchTiming when the reported value is one of the values {14,28,48} and when enableBeamSwitchTiming is not provided or the NZP-CSI-RS-ResourceSet is configured with the higher layer parameter trs-Info , aperiodic CSI-RS in a NZP-CSI-RS-ResourceSet configured with the higher layer parameter repetition set to 'off' or configured without the higher layer parameters repetition and trs-Info scheduled with offset larger than or equal to 48 when the UE provides beamSwitchTiming-r16 and enableBeamSwitchTiming is provided, aperiodic CSI-RS in a NZP-CSI-RS-ResourceSet configured with the higher layer parameter repetition set to 'on' scheduled with offset larger than or equal to the UE reported 

In [41]:
response = llm.invoke(prompt_input)

     

/home/lyus4/anaconda3/envs/rag_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


In [42]:
print(response)

Beam management refers to the process of controlling and optimizing the use of beams in a 5G wireless communication system. This includes tasks such as beamforming, beam switching, and beam allocation. Beamforming is the process of directing radio signals towards a specific user or group of users by using multiple antennas at the transmitter and receiver. Beam switching is the process of switching between different beams to improve signal quality or to avoid interference from other beams. Beam allocation is the process of assigning users to specific beams based on their location and the available beams in the system. Beam management is an important aspect of 5G wireless communication as it helps to improve the efficiency and performance of the system by optimizing the use of radio resources.

*Beam management is a process that involves controlling and optimizing the use of beams in a 5G wireless communication system.*

*Beamforming is the process of directing radio signals towards a sp

In [43]:
response_raw = llm.invoke("In 5G wireless communications, What is beam management?")
print(response_raw)

/home/lyus4/anaconda3/envs/rag_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


Beam management is a key technology in 5G wireless communications that allows for the efficient use of radio frequency (RF) resources and improved network performance. It involves the use of multiple antennas at the base station and the mobile device to create a beam of radio waves that is directed towards the device, rather than broadcasting in all directions. This allows for a more focused and efficient use of RF resources, which can lead to improved data rates, reduced latency, and increased capacity in the network. Beam management can also be used to improve the signal quality for devices that are located in areas with poor signal strength, by directing the beam towards the device and reducing interference from other sources.


In [54]:
# ── 依赖 ─────────────────────────────────────────────────────────────
from langchain_core.prompts   import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import (
    Runnable, RunnableLambda, RunnableWithMessageHistory, RunnablePassthrough
)
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory  # 或 RedisChatMessageHistory
# 本地 util / LLM / vectordb 与之前保持一致
from utils import format_context_grouped, expand_acronyms
# llm   = DeepSeekLLM(...)
# vs    = your VectorStore (...)

# ── 1. Prompt 模板 ──────────────────────────────────────────────
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a 5G wireless communication expert. "
     "Use the context and conversation history to answer the question. "
     "If neither provides a clear answer, respond exactly: **I don't know**.\n"
     "Explain clearly in numbered bullet points."),
    
    MessagesPlaceholder(variable_name="history"),  # 多轮历史插入

    ("user", 
     "<Context>\n{context}\n\n"
     "<Question>\n{input}")
])

# ── 2. 构造上下文（支持历史问句合并） ─────────────────────────────
def build_inputs(inputs: dict):
    question = inputs["input"]
    history_msgs = inputs.get("history", [])

    # 合并历史中所有 HUMAN 消息作为检索 query
    history_text = "\n".join([msg.content for msg in history_msgs if msg.type == "human"])
    combined_query = f"{history_text}\n{question}" if history_text else question
    q_exp = expand_acronyms(combined_query)

    print(f"\n[1] Expanded + Historical Query:\n{q_exp}")

    # 检索上下文
    top_k_results = vs.search(q_exp, k=3, score_mode="reciprocal")
    ctx = format_context_grouped(top_k_results, with_metadata=True, with_score=True)
    print(f"\n[2] Retrieved Context:\n{ctx}")

    # 拼接输入
    output = dict(inputs)  # 保留 input, history 字段
    output.update({"context": ctx, "input": question})  # 替换 input 为原始用户问题
    return output

context_retriever = RunnableLambda(build_inputs)

# ── 3. 构建完整对话链 ─────────────────────────────────────────────
base_chain: Runnable = (
    context_retriever |
    chat_prompt |
    llm |
    StrOutputParser()
)

# ── 4. 历史工厂（支持缓存）────────────────────────────────────────
history_store = {}

def history_factory(session_id: str):
    if session_id not in history_store:
        history_store[session_id] = InMemoryChatMessageHistory()
    return history_store[session_id]

# ── 5. 构建带历史支持的 chatbot ─────────────────────────────────
chatbot = RunnableWithMessageHistory(
    base_chain,
    history_factory,
    input_messages_key="input",      # 本轮问题字段
    history_messages_key="history"   # 多轮消息字段
)

# ── 6. 示例调用（支持多轮）──────────────────────────────────────
session_id = "user_42"  # 模拟一个用户 ID

# 第 1 轮
question1 = "What is beam management?"
print(f"\n[3] User Input: {question1}")
response1 = chatbot.invoke(
    {"input": question1},
    config={"configurable": {"session_id": session_id}}
)
print(f"[4] LLM Response:\n{response1}")

# 查看历史
print(f"\n[5] Current History:")
for msg in history_factory(session_id).messages:
    print(f"{msg.type.upper()}: {msg.content}")

# 第 2 轮
question2 = "And why is it important?"
print(f"\n[3] User Input: {question2}")
response2 = chatbot.invoke(
    {"input": question2},
    config={"configurable": {"session_id": session_id}}
)
print(f"[4] LLM Response:\n{response2}")

# 查看历史
print(f"\n[5] Current History:")
for msg in history_factory(session_id).messages:
    print(f"{msg.type.upper()}: {msg.content}")


/home/lyus4/anaconda3/envs/rag_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.



[3] User Input: What is beam management?

[1] Expanded + Historical Query:
What is beam management?

[2] Retrieved Context:
[Document: ../documents/3GPP_38/38214-i60.docx, Page: 1]
(score=0.495) to the UE reported threshold beamSwitchTiming when the reported value is one of the values {14,28,48} and when enableBeamSwitchTiming is not provided or the NZP-CSI-RS-ResourceSet is configured with the higher layer parameter trs-Info , aperiodic CSI-RS in a NZP-CSI-RS-ResourceSet configured with the higher layer parameter repetition set to 'off' or configured without the higher layer parameters repetition and trs-Info scheduled with offset larger than or equal to 48 when the UE provides beamSwitchTiming-r16 and enableBeamSwitchTiming is provided, aperiodic CSI-RS in a NZP-CSI-RS-ResourceSet configured with the higher layer parameter repetition set to 'on' scheduled with offset larger than or equal to the UE reported threshold beamSwitchTiming-r16 and enableBeamSwitchTiming is provided;

---



/home/lyus4/anaconda3/envs/rag_env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


[4] LLM Response:
Beam management refers to the process of controlling and optimizing the use of beams in a 5G wireless communication system. This includes tasks such as beamforming, beam switching, and beam allocation. The goal of beam management is to maximize the efficiency and performance of the system by directing radio signals towards specific users or devices and minimizing interference between different users or devices.

[5] Current History:
HUMAN: What is beam management?
AI: Beam management refers to the process of controlling and optimizing the use of beams in a 5G wireless communication system. This includes tasks such as beamforming, beam switching, and beam allocation. The goal of beam management is to maximize the efficiency and performance of the system by directing radio signals towards specific users or devices and minimizing interference between different users or devices.

[3] User Input: And why is it important?

[1] Expanded + Historical Query:
What is beam manag